In [54]:
import numpy as np
import matplotlib.pyplot as plt
import module

def compute_proxy_nodes_from_esn(esn, action_nodes, goal_nodes, num_proxy_nodes, trials=10, steps=500, discard=0, inverse=False, plot=False, figname = ""):  
    """
    Compute proxy nodes by selecting nodes that maximize temporal correlation with the goal nodes.
    Generates multiple random agent actions to test correlations.

    Parameters
    ----------
    esn : EchoStateNetwork
        Instance of the Echo State Network.
    action_nodes : array-like
        Indices of nodes controlled by the agent.
    goal_nodes : array-like
        Indices of the goal nodes.
    num_proxy_nodes : int
        Number of proxy nodes to select.
    trials : int, optional
        Number of random agent action sets to generate, by default 10.
    steps : int, optional
        Number of steps to run the ESN for each random action set, by default 500.
    discard : int, optional
        Number of initial steps to discard (transient period), by default 100.

    Returns
    -------
    np.ndarray
        Indices of the selected proxy nodes.
    """


    if plot:
        fig, axes = plt.subplots(2, 6, figsize=(50, 15))
        distance =  esn.distance(action_nodes, goal_nodes)
        fig.suptitle('Examples of random node value in 1 ESN, distance to the actions: '+ str(distance), fontsize=20)
        axes[0,0].set_ylabel('Value in the ESN', fontsize=20) 

    
    num_nodes = esn.n
    possible_nodes =  [i for i in range(num_nodes) if i not in action_nodes and i not in goal_nodes]
    run_data_base = np.zeros((trials, num_nodes))
    avg_goal_values = []
    avg_proxy_values = []
    for k in range(trials):
        # Generate random agent actions
        action_node_values = np.random.choice([-1.0, 0.0, 1.0], size=len(action_nodes))
        
        # Run the ESN with the random agent actions
        states = esn.run(steps=steps, discard=discard, agent_nodes=action_nodes, agent_values=action_node_values)
        run_data_base[k] =  np.mean(states[:, :], axis=0)

        # Compute the average values 
        avg_goal_values.append(np.mean(states[:, goal_nodes], axis=0))
        avg_proxy_values.append(np.mean(states[:, possible_nodes], axis=0))

        if k< 6 and plot:
            
            axes[0,k].set_xlabel('temporal steps', fontsize=20)
            axes[0,k].set_ylim(-1, 1)
            axes[0,k].plot(states[:, goal_nodes], label ='Goal Nodes', linestyle='--', linewidth=3)



    #print("run_data_base", run_data_base)
    # Compute correlation of each node with the goal values
    #print(run_data_base, np.corrcoef(run_data_base, rowvar= False),goal_nodes)
    correlations = np.mean(np.corrcoef(run_data_base, rowvar= False)[:, goal_nodes], axis=1)
    #print("correlations", correlations) 
    
    # filter nan values
    correlations = np.nan_to_num(correlations)

    # Sort nodes by correlation in descending order and select top nodes
    sorted_indices = [k for k in np.argsort(correlations)[::-1 + 2*inverse] if k in possible_nodes]
    proxy_nodes = sorted_indices[:num_proxy_nodes]
    #print('sorted correlations', [correlations[k] for k in proxy_nodes if k in possible_nodes] )
    #print("corr_before",correlations[proxy_nodes])

    #print("goal value", avg_goal_values)


    if plot:
        gs0 = axes[1, 0].get_gridspec()
        gs1 = axes[1, 3].get_gridspec()
        # remove the underlying Axes
        for ax in axes[-1, :]:
            ax.remove()
        axB0 = fig.add_subplot(gs1[1:, 0:3])
        axB1 = fig.add_subplot(gs1[1:, 3:6])
        axB0.set_title('Histogram of Node Values over '+ str(trials)+ ' Trials', fontsize=20)
        axB0.set_xlim(-1, 1)
        axB0.hist([k[0] for k in avg_goal_values], bins=50)
        axB1.set_xlim(-1, 1)
        axB1.hist([k for k in correlations], bins=50)
        axB1.set_title('Node Correlations with Goal Nodes', fontsize=20)
        fig.tight_layout()
        if figname != "":
            plt.savefig(figname + ".png")
            plt.close()
        """ plt.figure(figsize=(10, 5))
        plt.plot(states[:, proxy_nodes], label =['Proxy' + str(i) for i in range(num_proxy_nodes)], linestyle='-')
        plt.plot(states[:, goal_nodes], label ='Goal Nodes', linestyle='--', linewidth=3)
        plt.title('Node Correlations with Goal Nodes')
        plt.xlabel('temporal steps')
        plt.ylabel('Value in the ESN')
        plt.ylim(-1, 1)
        plt.legend()
        plt.grid()
        plt.show() """

    #print("goal base", np.mean(avg_goal_values))
    #print("example proxy ",[np.mean(states[:, node]) for node in proxy_nodes])
    goal_value = np.mean(avg_goal_values)
    proxy_value = np.mean(avg_proxy_values)

    return proxy_nodes, goal_value, proxy_value

In [73]:
# Example parameters
n = 400  # Total nodes
k = 5  # Number of agent-controlled nodes
goal_nodes = np.array([n//2+1])  # Indices of goal nodes
num_proxy_nodes = 5  # Number of proxy nodes to select
action_nodes = np.array([j for j in range(k)])  # Indices of agent-controlled nodes
trials = 1000  # Number of random actions to generate
discard = 00  # Number of steps to discard for transient period
steps = 50  # Number of steps to run the ESN
inverse = False  # Whether to select nodes with minimum correlation
# Initialize the Echo State Network
esn = module.EchoStateNetwork(n, spectral_radius=3, alpha = 0.5)


# Compute proxy nodes
proxy_nodes, goal_base_value, proxy_base_value = compute_proxy_nodes_from_esn(esn, action_nodes, goal_nodes, num_proxy_nodes, trials=trials, discard=discard, steps = steps, plot =True, inverse=inverse, figname = "parameter_exploration/test_proxy_nodes")
print("Selected proxy nodes:", proxy_nodes)
print("Goal base value:", goal_base_value)
print("Proxy base value:", proxy_base_value)

Selected proxy nodes: [np.int64(282), np.int64(115), np.int64(30), np.int64(202), np.int64(393)]
Goal base value: 0.0032748064395772803
Proxy base value: 0.00025316827826008307


In [ ]:
# create a dictionary with multiple value of a the parameters and try all the combinations  
param_grid = {
    'n': [40, 400],
    'k': [4, 16, 38],
    'num_proxy_nodes': [1],
    'trials': [100],
    'discard': [0, 500],
    'add_steps': [50, 500],
    'inverse': [False],
    'plot': [True],
    'spectral_radius': [1, 1.5],
    'alpha': [0.1,0.5],
    "plot": [True]
}


# creates a list of the parameters that will vary
varying_params = [a for a in param_grid.keys() if len(param_grid[a]) > 1]
# Create a list of all combinations of parameters
from itertools import product
param_combinations = list(product(*param_grid.values()))
print("param_combinations", len(param_combinations))
# Create a list of parameter names
param_names = list(param_grid.keys())
# Loop through all combinations of parameters
for params in param_combinations:
    # Create a dictionary of parameters
    param_dict = dict(zip(param_names, params))
    print("param_dict", param_dict)
    figure_name = 'parameter_exploration/ESN_'+ "".join([a +'_'+ str(b) +'_' for (a,b) in list(zip(param_names, params)) if a in varying_params]) + '.png'
    print("figure_name", figure_name)

    param_dict["figname"] = figure_name
    param_dict["steps"] = param_dict['add_steps'] + param_dict['discard']
    param_dict['action_nodes'] = np.array([j for j in range(param_dict['k'])])
    param_dict['goal_nodes'] = np.array([param_dict['n']-1])
    param_dict['esn'] = module.EchoStateNetwork(param_dict['n'], spectral_radius=param_dict['spectral_radius'], alpha = param_dict['alpha'])

    spectral_radius = param_dict['spectral_radius']
    alpha = param_dict['alpha']
    add_steps = param_dict['add_steps']
    n = param_dict['n']

    param_dict.pop('alpha')
    param_dict.pop('spectral_radius')
    param_dict.pop('add_steps')
    param_dict.pop('n')
    param_dict.pop('k')

    proxy_nodes, goal_base_value, proxy_base_value = compute_proxy_nodes_from_esn(**param_dict)
    
    
    # Unpack the parameters





param_combinations 2
param_dict {'n': 40, 'k': 38, 'num_proxy_nodes': 1, 'trials': 100, 'discard': 500, 'add_steps': 500, 'inverse': False, 'plot': True, 'spectral_radius': 1.5, 'alpha': 0.5}
figure_name parameter_exploration/ESN_alpha_0.5_.png
param_dict {'n': 40, 'k': 38, 'num_proxy_nodes': 1, 'trials': 100, 'discard': 500, 'add_steps': 500, 'inverse': False, 'plot': True, 'spectral_radius': 1.5, 'alpha': 1}
figure_name parameter_exploration/ESN_alpha_1_.png


In [69]:
[a +'_'+ str(b) +'_' for (a,b) in list(zip(param_names, params))]

['n_40_',
 'k_4_',
 'num_proxy_nodes_1_',
 'number_of_trials_100_',
 'discard_0_',
 'add_steps_50_',
 'inverse_False_',
 'plot_True_',
 'spectral_radius_1_',
 'alpha_0.1_']

In [ ]:
import numpy as np
from scipy.optimize import minimize, basinhopping
import matplotlib.pyplot as plt

def run_proxy_optimization_test(n, num_proxy_nodes, steps, discard, num_action_nodes,  trials, sample_for_agents, inverse=False):
    """

    """
    goal_start_values = []
    proxy_start_values = []
    goal_end_values = []
    proxy_end_values = []

    # Initialize the Echo State Network
    esn = module.EchoStateNetwork(n, spectral_radius=2, alpha=0.1)

    # Define goal and agent nodes
    goal_nodes = np.array([n-1])  # the goal node is the last node by permutation it is similar to a random pick 
    action_nodes = np.array([i for i in range(num_action_nodes)])  # First 10 nodes as agent-controlled nodes

    # Compute proxy nodes
    proxy_nodes, goal_base_value = compute_proxy_nodes_from_esn(esn, action_nodes, goal_nodes, num_proxy_nodes, 
                                            trials=trials, steps=steps, discard=discard, inverse=inverse)
    
    # define function to test an agent action 
    def test_agent_action(agent_action, action_value):
        """
        Test the average value of an action node if all other nodes are set to random values
        """
        action_node_values = [0.0 for i in range(num_action_nodes)]
        action_node_values[agent_action] = action_value

        proxy_samples    = []
        goal_samples    = []
        for j in range(sample_for_agents):
            for i in range(num_action_nodes):
                if i != agent_action:
                    action_node_values[i] = np.random.choice([-1.0, 0.0, 1.0])
            states = esn.run(steps=steps, discard=discard, action_nodes=action_nodes, action_node_values=action_node_values)
            proxy_samples.append(np.mean(states[discard:, proxy_nodes]))
            goal_samples.append(np.mean(states[discard:, goal_nodes]))
        return np.mean(proxy_samples), np.mean(goal_samples)


    action_node_values = [0.0 for i in range(num_action_nodes)]

    # test all possible agent values
    for i in range(num_action_nodes):
        for val in [-1.0, 1.0]:
            action_node_values[i] = val
            proxy_value, goal_value = test_agent_action(i, val)
    # Return the average difference in correlation
    return (np.mean(goal_end_values), np.mean(proxy_end_values), np.mean(goal_start_values), np.mean(proxy_start_values))

# Example usage
num_instances = 10
n = 500  # Total nodes
num_proxy_nodes = 1
steps = 500
discard = 0
trials = 10
num_action_nodes = 10
sample_for_agents = 10
inverse = False

average_goal_value, average_proxy_value, average_goal_base, average_proxy_base = run_proxy_optimization_test(num_instances, n, num_proxy_nodes, steps, discard, num_action_nodes, trials, sample_for_agents, inverse)
print("Average value of the goal base", average_goal_base)
print("Average value of the goal", average_goal_value)

print("Average value of the proxy base", average_proxy_base)
print("Average value of the proxy", average_proxy_value)



sorted correlations [np.float64(0.8884256063344389)]
res [-0.87586425  0.23393104 -0.85282158 -0.80155004 -0.18991463 -0.50793166
  0.86592433 -0.95189146  0.08697777  1.        ]
proxbase, goalbase, proxend, goalend 0.013273841217008147 0.10660975405853844 0.6364214700017361 0.43780479792340954
sorted correlations [np.float64(0.9490633132556473)]


KeyboardInterrupt: 